In [1]:
import os, csv, random

from __future__ import division
from matplotlib import  pyplot as plt
from memory_profiler import profile
from scipy import stats
from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import scipy.io as sio
import seaborn as sns

%matplotlib inline

# Generate score for videos, and find all possible pairs

In [2]:
video_num = 30

video_score = np.random.uniform(-5,5,video_num)
resolution = 0.1
video_score = np.round(video_score/resolution)*resolution

pairs_truth = []

for i in range(len(video_score)-1):
    for j in range(i+1, len(video_score)):
        if video_score[i] > video_score[j]:
            pairs_truth.append((i,j))
        else:
            pairs_truth.append((j,i))
            
total_pairs = len(pairs_truth)
print 'Total number of pairs for %d videos is: ' %video_num, total_pairs

Total number of pairs for 30 videos is:  435


# SGD
Assuming the pair $(w_i,w_j)$ means $w_i > w_j$, this function calculates $\frac{\partial L}{\partial w_i} = \frac{exp(w_j-w_i)}{1+exp(w_j-w_i)}$, where $w_i$ represent the weight to be taken derivative of. Return value is the gradient of the log likelihood function with respect to $w_i$

In [8]:
def calc_gradient(pairs, w, w_i):
    gradient = 0
    
    for pair in pairs:
        if w_i == pair[0]:
            out = -1
        elif w_i == pair[1]:
            out = 1  
        else:
            out = 0
            
        gradient -= out/(1/(np.exp(w[pair[1]]-w[pair[0]]))+1)
    
    return gradient

$w_{old}$ saves the original values of the gradient, so that later when updating the values, the values to be updated are still using the original value

In [10]:
def update_gradient(pairs, w, eta):
    w_old = w
    for i in range(len(w)):
        w[i] += eta*calc_gradient(pairs, w_old, i)
    return w

In [18]:
def likelihood(pairs, w):
    l = 1
    for pair in pairs:
        l += np.log(1+np.exp(w[pair[1]]-w[pair[0]]))
    return l

In [19]:
w = np.zeros(30)
pairs = pairs_truth
results = []
for i in range(10000):
    
    w = update_gradient(pairs, w, 0.01)
    if i % 500 ==0:
#         print w
        print likelihood(pairs_truth, w)
    results.append(w[0])
    

280.911721014
35.8156555211
26.5535403357
22.1919668482
19.4908374881
17.5960079407
16.1671714304
15.0374277632
14.1136934517
13.3392654908
12.6772976688
12.1026330975
11.5974103694
11.1485385899
10.7461674325
10.3827188481
10.0522514892
9.75003039168
9.47222774914
9.21570995382


In [22]:
true_order = np.array(video_score).argsort()
true_ranks = true_order.argsort()

temp_o = np.array(w).argsort()
temp_r = temp_o.argsort()

print true_ranks
print temp_r
print w

[15 14 21 29  4  8  0 28  6  9  1 20 16 17 25 10 22 11 19 26 18  7 12 27 24
  3  2  5 23 13]
[15 14 20 28  4  8  0 29  5  9  1 21 16 17 25 10 22 11 19 26 18  7 12 27 24
  3  2  6 23 13]
[  0.61503281  -0.61658997   6.907792    19.80744364 -14.07631589
  -8.23463522 -22.53531545  22.53453472 -12.50247263  -6.90929083
 -19.80883775   8.23328938   1.84898605   3.08999934  14.07504652
  -5.61505851   9.59901248  -4.34456394   5.61361208  15.77179651
   4.34309981  -9.60040282  -3.09156515  17.64681377  12.50126534
 -15.77313287 -17.6481498  -11.01793091  11.01662265  -1.85057937]


# Generate Data using Generative function 

In [13]:
video_score

array([-0.5, -1. ,  1.5,  4.4, -3.7, -2.7, -4.7,  4.4, -3.4, -2.5, -4.1,
        1.5, -0.4, -0.2,  3.2, -2.5,  1.7, -2. ,  1.1,  3.8,  0.7, -3. ,
       -1.3,  4.2,  3.1, -3.9, -4. , -3.4,  2.2, -1.1])